In [33]:
# Load dependencies and set constants
from pyflink.datastream.functions import ProcessFunction
from pyflink.common.serialization import SimpleStringSchema
from pyflink.common.typeinfo import Types
from pyflink.datastream import StreamExecutionEnvironment, TimeCharacteristic
from pyflink.datastream.connectors import FlinkKafkaConsumer
from matplotlib import pyplot as plt
import time
import sys


kafka_props = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'twitter-consumers'
}

KAFKA_TOPIC = "twitter-stream"
KAFKA_CONNECTOR_JAR = "file:///home/ubuntu/flink-sql-connector-kafka_2.11-1.12.2.jar"
# Number of tweets in the dataset (10 MB)
NUMBER_OF_TWEETS = 14484
# Number of times the dataset is produced by Kafka
NUMBER_OF_PRODUCTIONS = 10

In [34]:
latencies = []
records_received = 0

def collect_stats():
    plt.clf()
    plt.plot(latencies)
    plt.savefig('latency_plot.png')


class MyProcessFunction(ProcessFunction):

    def process_element(self, value, ctx: 'ProcessFunction.Context'):
        latency = (time.time() * 1000) - ctx.timestamp()
        #result = "Latency: {}   Record-Number: {}   Total Number of Records: {}".format(str(latency), str(len(latencies)), str(NUMBER_OF_TWEETS * NUMBER_OF_PRODUCTIONS))
        result = str(latency)
        yield result
        # global might not be ideal here (parallelism possible?) but currently nothing else comes to mind
        global records_received
        records_received += 1
        latencies.append(latency)
        if records_received == NUMBER_OF_TWEETS * NUMBER_OF_PRODUCTIONS:
            collect_stats()


env = StreamExecutionEnvironment.get_execution_environment()
'''
An execution environment defines a default parallelism for all operators, data sources,
and data sinks it executes
'''
env.set_parallelism(100)
env.set_stream_time_characteristic(TimeCharacteristic.EventTime)
# Add the Kafka Connector Dependency
env.add_jars(KAFKA_CONNECTOR_JAR)
env.add_classpaths(KAFKA_CONNECTOR_JAR)

kafka_consumer = FlinkKafkaConsumer("twitter-stream", SimpleStringSchema(), kafka_props)

stream = env.add_source(kafka_consumer)
stream.process(MyProcessFunction(), output_type=Types.STRING()).print()

env.execute()


'''
Weird stuff:
    - Sometimes more records than emitted by Kafka (sometimes also less)
    - print output sometimes was "Latency: {}   Record-Number: {}   Total Number of Records: {}".format(str(latency), str(len(latencies)), str(NUMBER_OF_TWEETS * NUMBER_OF_PRODUCTIONS))
      and sometimes str(latency)!?!
    - what value for parallelism?
    
Notes:
    - Flink parallelism: https://stackoverflow.com/questions/61139187/what-is-the-difference-between-parallelism-and-parallel-computing-in-flink
'''




ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:40547)
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 958, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1096, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:40547)